<a href="https://colab.research.google.com/github/saifullahAnsari0001/Email-automation-task/blob/main/email_automation_reply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip -q install crewai langchain-groq duckduckgo-search
!pip install -qU  langchain-core
!pip install -qU langchain-community
!pip install -qU  'crewai[tools]'
!pip install -qU composio_langchain
!pip install sentence-transformers
!pip install langchain_huggingface

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [21]:
import sqlite3
conn = sqlite3.connect('film_equipment.db')
cursor = conn.cursor()

cursor.execute('''
      CREATE TABLE equipment (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          item TEXT NOT NULL,
          category TEXT NOT NULL,
          price REAL,
          availability BOOLEAN
      )
''')

equipment_data = [
      ("Cannon XA70", "Camera", 2000.0, True),
      ("Sony A7 III", "Camera", 1800.0, False),
      ("ARRI Alexa", "Camera", 1500.0, True),
      ("Nikon D850", "Camera", 1300.0, False),
      ("Canon EF 24-70mm f/2.8L II USM Lens", "Lens", 200.0, False),
      ("Fotasy F1.6 Prime", "Lens", 30.0, True),
      ("Olympus OM-D E-M1 Mark III", "Lens", 150.0, True),
      ("Sony FE 70-200mm", "Lens", 2000.0, False),
      ("18K HMI Light", "Lighting", 300.0, False),
      ("Godox LDX50R", "Lighting", 200.0, True),
      ("Sennheiser 416 Shotgun Microphone", "Sound", 150.0, False),
      ("Movo WMX-1", "Sound", 40.0, False),
      ("Zoom H1essential", "Sound", 100.0, True),
      ("Sirui SH15", "Tripod", 150.0, True),
      ("Magnus DWF-2", "Tripod", 250.0, False),
      ("Zhiyun Crane-M2", "Gimbal", 200.0, True),
      ("Zhiyun Crane-M2", "Gimbal", 279.0, False),
      ("Final Cut Pro", "Software", 299.0, True),
      ("Adobe Premiere Pro", "Software", 799.0, False)
  ]

cursor.executemany('INSERT INTO equipment (item, category, price, availability) VALUES (?, ?, ?, ?)', equipment_data)

conn.commit()
conn.close()

In [22]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from langchain_groq import ChatGroq
from composio_langchain import App, ComposioToolSet

In [23]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["COMPOSIO_API_KEY"] = userdata.get("COMPOSIO_API_KEY")

In [24]:
GROQ_LLM = ChatGroq(
            model="llama3-70b-8192"
        )

In [29]:
!composio apps update
toolset = ComposioToolSet()
code_interpreter_tools = toolset.get_tools([App.CODEINTERPRETER])
sql_tools = toolset.get_tools([App.SQLTOOL])

⚠️ Apps does not require update
⚠️ Tags does not require update
⚠️ Actions does not require update
⚠️ Triggers does not require update


/usr/local/lib/python3.10/dist-packages/composio/client/collections.py:884: UserWarning: Using all the actions of an app is not recommended. Please use tags to filter actions or provide specific actions. We just pass the important actions to the agent, but this is not meant to be used in production. Check out https://docs.composio.dev/sdk/python/actions for more information.
  warnings.warn(


In [25]:
from crewai_tools import PDFSearchTool

In [27]:
rag_tool = PDFSearchTool(pdf='Film_Equipment_FAQs.pdf',
    config=dict(
        llm=dict(
            provider="groq",
            config=dict(
                model="llama3-70b-8192",
            ),
        ),
        embedder=dict(
            provider="huggingface",
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


In [48]:
email_content = """HI there, \n
I am interested in the Sony A7 III and would like to know if it is currently in stock.\n
Could you please provide me with the availability status and pricing details? \n

Thank you for your prompt attention to this matter.

Thanks,
George
"""

# **SQL agent**

In [10]:
sql_agent = Agent(
    role="SQL Agent",
    goal=(
        "Run SQL queries to handle inquiry emails for a film equipment rental service. "
        "Check the database for item availability. "
        "If available, retrieve and provide the item's name and price. "
        "If not available, suggest similar category available items that are in stock."
    ),
    backstory=(
        "You are an agent that helps users run SQL queries. "
        "Connect to the local SQLite DB at connection string = film_equipment.db. "
        "Analyze the tables by listing all the tables and columns, and exploring distinct values for each column. "
        "Once you have a good understanding, construct queries to check item availability, retrieve item prices, and find similar items."
    ),
    verbose=True,
    tools=sql_tools,
    llm=GROQ_LLM,
    allow_delegation=True,
)

sql_task = Task(
    description="Run SQL queries to achieve a task - {email_content}",
    expected_output="SQL queries executed successfully. The result of the task is returned - {email_content}",
    agent=sql_agent,
)

# **Categorizer agent**

In [58]:
categorizer_agent = Agent(
      role='Email Categorizer Agent',
      goal="""Take in an email from a human that has emailed our company email address and categorize it \
      into one of the following categories: \
      inquiry - used when someone is asking for information about pricing or product features \
      review - used when someone is providing a review, whether positive or negative \
      assistance_request - used when someone is asking for help with an issue or requesting assistance \
     """,
      backstory="""You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way.""",
      llm=GROQ_LLM,
      verbose=True,
      allow_delegation=False,
      max_iter=5,
      memory=True,
      # step_callback=lambda x: print_agent_output(x, "Email Categorizer Agent"),
        )

categorize_task = Task(
    description=f"""Conduct a comprehensive analysis of the email provided and categorize it into \
    one of the following categories:
    inquiry - used when someone is asking for information about pricing or product features \
    review - used when someone is providing a review, whether positive or negative \
    assistance_request - used when someone is asking for help with an issue or requesting assistance \

    EMAIL CONTENT:\n\n {email_content} \n\n
    Output a single category only""",
    expected_output="""A single category for the type of email from the types ('inquiry', 'review', 'assistance_request') \
    e.g.:
    'inquiry' """,
    output_file="email_category.txt",
    agent=categorizer_agent
)

# **Review Handler agent**

In [41]:
review_handling_agent = Agent(
      role='Review Handling Agent',
      goal="""Take in an email from a human that has emailed our company email address and the category \
      'review' given by the categorizer agent, then:
      - For positive reviews, thank the sender and encourage them to share their experience on social media.
      - For negative reviews, assure them that we value their feedback, escalate to the CRM system for follow-up with a phone call from customer service, and offer a gift voucher in the reply.""",
      backstory="""You are a master at handling reviews and know how to respond to both positive and negative feedback to maintain and enhance customer relationships.""",
      llm=GROQ_LLM,
      verbose=True,
      allow_delegation=False,
      max_iter=5,
      memory=True,
      # step_callback=lambda x: print_agent_output(x, "Review Handling Agent"),
  )

review_handling_task = Task(
        description=f"""Handle the following review:
        REVIEW CONTENT:\n\n {email_content} \n\n
        If the review is positive:
        - Thank the sender.
        - Encourage them to share their experience on social media.

        If the review is negative:
        - Assure them that we value their feedback.
        - Escalate to the CRM system for follow-up with a phone call from customer service.
        - Offer a gift voucher in the reply.""",
        expected_output="""A response to the review that either:
        - Thanks the sender and encourages them to share their experience on social media (for positive reviews).
        - Assures the sender that we value their feedback, mentions escalation to the CRM system, and offers a gift voucher (for negative reviews).""",
        output_file="review_response.txt",
        agent=review_handling_agent
  )


# **Assistance request handling**

In [54]:
assistance_request_handling_agent = Agent(
            role='Assistance Request Handling Agent',
            goal="""Take in an email from a human that has emailed our company email address with an assistance request. The category \
            'assistance_request' given by the categorizer agent, then:
            - If a suitable solution is found, provide it in the reply without referencing the search process or any internal documents.
            - If no solution is found, escalate the issue to customer service.""",
            backstory="""You are a master at handling assistance requests and can efficiently find solutions or escalate issues to ensure customer satisfaction.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=10,
            memory=True,
            tools=[rag_tool],
            # step_callback=lambda x: print_agent_output(x, "Assistance Request Handling Agent"),
        )

assistance_request_handling_task =  Task(
            description=f"""Handle the following assistance request:
            REQUEST CONTENT:\n\n {email_content} \n\n
            - If a suitable solution is found, provide it in the reply.
            - If no solution is found, escalate the issue to customer service.""",
            expected_output="""A response to the assistance request that either:
            - Provides a solution to the reported issue.
            - If no solution is found, mentions escalation to customer service for further assistance.""",
            output_file="assistance_response.txt",
            agent=assistance_request_handling_agent
        )

# **Automatic Email Reply System for Film Equipment Rental Service**

In [54]:
class EmailAgents:
    # Categorizer Agent
    def make_categorizer_agent(self):
        return Agent(
            role='Email Categorizer Agent',
            goal="""Categorize incoming emails into one of the following categories:
            - inquiry: When someone is asking for information about pricing or product features
            - review: When someone is providing feedback, whether positive or negative
            - assistance_request: When someone is asking for help with an issue or requesting assistance""",
            backstory="""You excel at understanding the intent behind customer emails and categorizing them appropriately.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

    # SQL Agent for Handling Inquiries
    def make_sql_agent(self):
        return Agent(
            role='SQL Agent',
            goal="""Take in an email from a human that has emailed our company email address with an inquiry request. The category \
            'inquiry' given by the categorizer agent, then:
             Run SQL queries to handle inquiry emails for a film equipment rental service,
             Check the database for item availability,
             If available, retrieve and provide the item's name and price,
             If not available, suggest similar category available items that are in stock.""",
            backstory="""You are an agent that helps users run SQL queries,
            Connect to the local SQLite DB at connection string = film_equipment.db,
            Analyze the tables by listing all the tables and columns, and exploring distinct values for each column,
            Once you have a good understanding, construct queries to check item availability, retrieve item prices, and find similar items.""",
            verbose=True,
            tools=sql_tools,
            llm=GROQ_LLM,
            allow_delegation=True,
        )

    # Review Handling Agent
    def make_review_handling_agent(self):
        return Agent(
            role='Review Handling Agent',
            goal="""Take in an email from a human that has emailed our company email address with review. The category \
            'review' given by the categorizer agent, then:
            Respond to review emails based on their sentiment:
            - For positive reviews, thank the sender and encourage them to share their experience on social media
            - For negative reviews, assure the sender that their feedback is valued, escalate to CRM for follow-up, and offer a gift voucher""",
            backstory="""You specialize in crafting responses to both positive and negative reviews to enhance customer satisfaction.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

    # Assistance Request Handling Agent
    def make_assistance_request_handling_agent(self):
        return Agent(
            role='Assistance Request Handling Agent',
            goal="""Take in an email from a human that has emailed our company email address with an assistance request. The category \
            'assistance_request' given by the categorizer agent, then:
            - If a suitable solution is found, provide it in the reply without referencing the search process or any internal documents.
            - If no solution is found, escalate the issue to customer service.""",
            backstory="""You are skilled at finding solutions to customer issues or escalating them to ensure resolution.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=10,
            memory=True,
            tools=[rag_tool],
        )

    # Draft Email Writer Agent
    def make_draft_email_writer_agent(self):
        return Agent(
            role='Draft Email Writer Agent',
            goal="""Draft a final reply email based on the categorized email and the response from the corresponding agent (inquiry, review, or assistance request)""",
            backstory="""You excel at drafting professional and coherent email responses based on the provided context and agent responses.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )




In [55]:
class EmailTasks:
    # Categorize Email Task
    def categorize_email(self, email_content):
        return Task(
            description=f"""Categorize the email into one of the following categories:
            - inquiry: Asking for information about pricing or product features
            - review: Providing feedback (positive or negative)
            - assistance_request: Requesting help or assistance

            EMAIL CONTENT:\n\n {email_content} \n\n
            Output a single category only""",
            expected_output="""A single category from the types ('inquiry', 'review', 'assistance_request'). E.g.:
            'inquiry'""",
            output_file="email_category.txt",
            agent=categorizer_agent
        )

    # SQL Task for Inquiries
    def sql_task(self, email_content):
        sql_agent = EmailAgents().make_sql_agent()
        return Task(
            description=f"""Run SQL queries based on the email content to handle inquiry:

            EMAIL CONTENT:\n\n {email_content} \n\n
            Only provide the info needed DONT try to write the email""",
            expected_output="SQL queries executed successfully. The result of the task is returned - {email_content}",
            context = [categorize_email],
            agent=sql_agent,

        )

    # Review Handling Task
    def review_handling_task(self, email_content):
        review_handling_agent = EmailAgents().make_review_handling_agent()
        return Task(
            description=f"""Handle the following review:
            REVIEW CONTENT:\n\n {email_content} \n\n
            Only provide the info needed DONT try to write the email
            For positive reviews:
            - Thank the sender
            - Encourage sharing on social media

            For negative reviews:
            - Assure the sender that feedback is valued
            - Escalate to CRM for follow-up
            - Offer a gift voucher""",
            expected_output="""A response that:
            - Thanks the sender and encourages social media sharing (positive reviews)
            - Assures feedback is valued, mentions CRM escalation, and offers a gift voucher (negative reviews)""",
            context = [categorize_email],
            agent=review_handling_agent
        )

    # Assistance Request Handling Task
    def assistance_request_handling_task(self, email_content):
        assistance_request_handling_agent = EmailAgents().make_assistance_request_handling_agent()
        return Task(
            description=f"""Handle the following assistance request:
            REQUEST CONTENT:\n\n {email_content} \n\n
            Only provide the info needed DONT try to write the email
            - Provide a solution if available
            - If no solution is found, escalate to customer service""",
            expected_output="""A response that:
            - Provides a solution (if found)
            - Escalates to customer service if no solution is available""",
            context = [categorize_email],
            agent=assistance_request_handling_agent
        )

    # Draft Final Email Task
    def draft_email(self, email_content, category, *agent_response):
        draft_email_writer_agent = EmailAgents().make_draft_email_writer_agent()
        return Task(
            description=f"""Draft a final reply email based on the categorized email and the response generated by the corresponding agent:
            CATEGORY: {category}
            ORIGINAL EMAIL CONTENT:\n\n {email_content} \n\n
            response_details = "\n\n".join(agent_responses)""",
            expected_output="""A professionally drafted email response ready to be sent to the customer""",
            context = [categorize_email, sql_task, review_handling_task, assistance_request_handling_task],
            output_file="final_reply_email.txt",
            agent=draft_email_writer_agent
        )


In [56]:
agents = EmailAgents()
tasks = EmailTasks()

## Agents
categorizer_agent = agents.make_categorizer_agent()
sql_agent = agents.make_sql_agent()
review_handling_agent = agents.make_review_handling_agent()
assistance_request_handling_agent = agents.make_assistance_request_handling_agent()
draft_email_writer_agent = agents.make_draft_email_writer_agent()

## Tasks
categorize_email = tasks.categorize_email(email_content)
sql_task = tasks.sql_task(email_content)
review_handling_task = tasks.review_handling_task(email_content)
assistance_request_handling_task = tasks.assistance_request_handling_task(email_content)
draft_email = tasks.draft_email(email_content, categorize_email, sql_task, review_handling_task, assistance_request_handling_task)

In [57]:
crew = Crew(
    agents=[categorizer_agent, sql_agent, review_handling_agent, assistance_request_handling_agent, draft_email_writer_agent],
    tasks=[categorize_email, sql_task, review_handling_task, assistance_request_handling_task, draft_email],
    verbose=2,
    # process=Process.sequential,
    full_output=True,
    share_crew=False,
)

In [58]:
results = crew.kickoff()
print(results)

 [2024-07-28 20:11:25][DEBUG]: == Working Agent: Email Categorizer Agent
 [2024-07-28 20:11:25][INFO]: == Starting Task: Categorize the email into one of the following categories:
            - inquiry: Asking for information about pricing or product features
            - review: Providing feedback (positive or negative)
            - assistance_request: Requesting help or assistance

            EMAIL CONTENT:

 HI there, 

I am interested in the Sony A7 III and would like to know if it is currently in stock.

Could you please provide me with the availability status and pricing details? 


Thank you for your prompt attention to this matter.

Thanks,
George
 


            Output a single category only


> Entering new CrewAgentExecutor chain...
I understand the importance of this task, and I'm confident in my ability to categorize this email accurately.

Thought: I now can give a great answer

Final Answer: inquiry

> Finished chain.
 [2024-07-28 20:11:25][DEBUG]: == [Email Categoriz